In [2]:
import datetime
from ib_insync import *
import pandas as pd

import sys
from pathlib import Path

# Get the parent directory of the current notebook's directory
project_root = Path.cwd().parent

# Add the project root to the system path
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

try:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db
except:
    from strategy_manager.strategies.short_vix import Strategy
    from broker.connection import connect_to_IB
    from broker.trademanager import TradeManager
    from broker.portfolio import PortfolioManager
    from data_and_research import ac, initialize_db

In [3]:
ib = connect_to_IB(clientid=3)

# Develop a method to retrieve the current portfolio as dataframe and store it in arcticdb

In [4]:
pm = PortfolioManager(ib)

In [5]:
df1 = pm.get_positions_from_ib()
df1

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account
0,2024-02-02 11:19,EWT,STK,200.0,0.087705,44.256828,45.129135,0.019710,,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-02,9025.83,174.46,USD,0.0,U7706434
1,2024-02-02 11:19,IAU,STK,300.0,0.113254,38.454255,38.849998,0.010291,,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-02-02,11655.00,118.72,USD,0.0,U7706434
2,2024-02-02 11:19,IAU,Call 39.0 20240216,-2.0,-0.000708,23.350200,0.364405,-0.560607,,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-02-02,-72.88,-26.18,USD,0.0,U7706434
3,2024-02-02 11:19,INDA,STK,100.0,0.049072,49.712499,50.500000,0.015841,,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-02-02,5050.00,78.75,USD,0.0,U7706434
4,2024-02-02 11:19,IUSQ,STK,100.0,0.068798,65.602785,70.800995,0.079238,,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-02-02,7080.10,519.82,EUR,0.0,U7706434
5,2024-02-02 11:19,IWM,STK,100.0,0.190164,199.616920,195.698914,-0.019628,,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-02-02,19569.89,-391.80,USD,0.0,U7706434
6,2024-02-02 11:19,IWM,Call 199.0 20240216,-1.0,-0.002322,183.368900,2.389160,-0.302926,,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-02-02,-238.92,-55.55,USD,0.0,U7706434
7,2024-02-02 11:19,PRX,STK,250.0,0.068870,26.561554,28.350000,0.067332,,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-02-02,7087.50,447.11,EUR,0.0,U7706434
8,2024-02-02 11:19,SOXX,Put 525.0 20240216,-1.0,-0.000497,699.944800,0.511683,0.926897,,"Option(conId=653437229, symbol='SOXX', lastTra...",None,2024-02-02,-51.17,648.78,USD,0.0,U7706434
9,2024-02-02 11:19,TLT,STK,100.0,0.095102,99.051625,97.870003,-0.011929,,"Stock(conId=15547841, symbol='TLT', right='0',...",None,2024-02-02,9787.00,-118.16,USD,0.0,U7706434


In [10]:
base =  [entry.currency for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue"][0]
fx_list = set(df1['currency'].to_list())
fx_pairs = [Forex(base+fx) for fx in fx_list if base != fx]
fx_pairs
ib.reqMarketDataType(4)
ib.qualifyContracts(*fx_pairs)
fx_dict = {}
fx_dict[base] = 1
for fx in fx_pairs:
    
    price = ib.reqMktData(fx, '', False, False)
    ib.sleep(0.2)
    mktPrice = price.marketPrice()
    fx_dict[fx.currency] = mktPrice

fx_dict



{'EUR': 1, 'USD': 1.088705}

In [19]:
fx_cache = {}

def convert_marketValue_to_base(df):
    """
    Args:
        df (pd.DataFrame): DataFrame containing the 'marketValue' column.

    Returns:
        pd.DataFrame: The DataFrame with the 'marketValue_base' column
    """
    base_currency =  [entry.currency for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue"][0]
    # Use a pandas function for efficiency and clarity
    fx_rates = df['currency'].map(lambda c: get_fx_rate(c, base_currency))

    # Avoids unnecessary calculations if all currencies are already the base
    if fx_rates.unique().size > 1:
        df['marketValue_base'] = df['marketValue'] / fx_rates
    else:
        df['marketValue_base'] = df['marketValue']

    return df

def get_fx_rate(currency, base_currency):
    """Retrieves the FX rate from a cached dictionary or live IB request.

    Args:
        currency (str): The currency to convert from.
        base_currency (str): The base currency to convert to.

    Returns:
        float: The FX rate (currency / base_currency).
    """

    global fx_cache  # Use a global variable for caching

    if (currency, base_currency) not in fx_cache:
        if currency == base_currency:
            fx_cache[(currency, base_currency)] = 1.0
        else:
            fx_pair = Forex(base_currency+currency)
            ib.reqMarketDataType(4)  # Ensure market data type is set
            ib.qualifyContracts(fx_pair)
            price = ib.reqMktData(fx_pair, '', False, False)
            ib.sleep(0.2)  # Wait for data
            fx_cache[(currency, base_currency)] = price.marketPrice()

    return fx_cache[(currency, base_currency)]

In [21]:
df2 = convert_marketValue_to_base(df1)
df2

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account,marketValue_base
0,2024-02-02 11:19,EWT,STK,200.0,0.087705,44.256828,45.129135,0.019710,,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-02,9025.83,174.46,USD,0.0,U7706434,8292.407977
1,2024-02-02 11:19,IAU,STK,300.0,0.113254,38.454255,38.849998,0.010291,,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-02-02,11655.00,118.72,USD,0.0,U7706434,10707.936552
2,2024-02-02 11:19,IAU,Call 39.0 20240216,-2.0,-0.000708,23.350200,0.364405,-0.560607,,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-02-02,-72.88,-26.18,USD,0.0,U7706434,-66.957908
3,2024-02-02 11:19,INDA,STK,100.0,0.049072,49.712499,50.500000,0.015841,,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-02-02,5050.00,78.75,USD,0.0,U7706434,4639.646468
4,2024-02-02 11:19,IUSQ,STK,100.0,0.068798,65.602785,70.800995,0.079238,,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-02-02,7080.10,519.82,EUR,0.0,U7706434,7080.100000
5,2024-02-02 11:19,IWM,STK,100.0,0.190164,199.616920,195.698914,-0.019628,,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-02-02,19569.89,-391.80,USD,0.0,U7706434,17979.677430
6,2024-02-02 11:19,IWM,Call 199.0 20240216,-1.0,-0.002322,183.368900,2.389160,-0.302926,,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-02-02,-238.92,-55.55,USD,0.0,U7706434,-219.505809
7,2024-02-02 11:19,PRX,STK,250.0,0.068870,26.561554,28.350000,0.067332,,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-02-02,7087.50,447.11,EUR,0.0,U7706434,7087.500000
8,2024-02-02 11:19,SOXX,Put 525.0 20240216,-1.0,-0.000497,699.944800,0.511683,0.926897,,"Option(conId=653437229, symbol='SOXX', lastTra...",None,2024-02-02,-51.17,648.78,USD,0.0,U7706434,-47.012022
9,2024-02-02 11:19,TLT,STK,100.0,0.095102,99.051625,97.870003,-0.011929,,"Stock(conId=15547841, symbol='TLT', right='0',...",None,2024-02-02,9787.00,-118.16,USD,0.0,U7706434,8991.726729


In [6]:
df1['marketValue'] = df1.apply(lambda x: pm.convert_to_base_currency(x.marketValue,x.currency), axis=1)
df1

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,trade,open_dt,marketValue,unrealizedPNL,currency,realizedPNL,account
0,2024-02-01 17:37,EWT,STK,200.0,0.087334,44.256828,44.793907,0.012135,,"Stock(conId=253190576, symbol='EWT', right='0'...",None,2024-02-01,8254.657698,107.42,USD,0.0,U7706434
1,2024-02-01 17:37,IAU,STK,300.0,0.113987,38.454255,38.976002,0.013568,,"Stock(conId=474829650, symbol='IAU', right='0'...",None,2024-02-01,10773.695995,156.52,USD,0.0,U7706434
2,2024-02-01 17:37,IAU,Call 39.0 20240216,-2.0,-0.000853,23.350200,0.437316,-0.872857,,"Option(conId=671974250, symbol='IAU', lastTrad...",None,2024-02-01,-80.585271,-40.76,USD,0.0,U7706434
3,2024-02-01 17:37,INDA,STK,100.0,0.048699,49.712499,49.956001,0.004898,,"Stock(conId=101484826, symbol='INDA', right='0...",None,2024-02-01,4602.966922,24.35,USD,0.0,U7706434
4,2024-02-01 17:37,IUSQ,STK,100.0,0.068454,65.602785,70.220001,0.070381,,"Stock(conId=239363241, symbol='IUSQ', right='0...",None,2024-02-01,7022.000000,461.72,EUR,0.0,U7706434
5,2024-02-01 17:37,IWM,STK,100.0,0.187765,199.616920,192.610001,-0.035102,,"Stock(conId=9579970, symbol='IWM', right='0', ...",None,2024-02-01,17747.003160,-700.69,USD,0.0,U7706434
6,2024-02-01 17:37,IWM,Call 199.0 20240216,-1.0,-0.001337,183.368900,1.371862,0.251857,,"Option(conId=637114226, symbol='IWM', lastTrad...",None,2024-02-01,-126.406280,46.18,USD,0.0,U7706434
7,2024-02-01 17:37,PRX,STK,250.0,0.067887,26.561554,27.855463,0.048714,,"Stock(conId=382625193, symbol='PRX', right='0'...",None,2024-02-01,6963.870000,323.48,EUR,0.0,U7706434
8,2024-02-01 17:37,SOXX,Put 525.0 20240216,-1.0,-0.000849,699.944800,0.870475,0.875637,,"Option(conId=653437229, symbol='SOXX', lastTra...",None,2024-02-01,-80.208237,612.90,USD,0.0,U7706434
9,2024-02-01 17:37,TLT,STK,100.0,0.095831,99.051625,98.303909,-0.007549,,"Stock(conId=15547841, symbol='TLT', right='0',...",None,2024-02-01,9057.595916,-74.77,USD,0.0,U7706434


In [9]:
df1.iloc[0].currency 
df1.iloc[0].marketValue

pm.convert_to_base_currency(df1.iloc[0]['% of nav'],df1.iloc[0].currency )


NameError: name 'df1' is not defined

In [12]:
# Define the currency pair
forex_pair = Forex('EURUSD')

# Request market data
market_data = ib.reqMktData(forex_pair, '', False, False)

# Wait for the data to be filled
ib.sleep(1)

market_data.

<bound method Ticker.marketPrice of Ticker(contract=Forex('EURUSD', exchange='IDEALPRO'), time=datetime.datetime(2024, 1, 28, 20, 46, 42, 131427, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=-1.0, bidSize=0.0, ask=-1.0, askSize=0.0, close=1.0847, halted=0.0)>

In [13]:

latest_positions = df.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-26 17:00:33.626296,200.0,0.088367,44.256828,45.166000,0.020543,"Stock(conId=253190576, symbol='EWT', right='0'...",9033.20,181.83,0.0,U7706434
1,IAU,,Call 39.0 20240216,2024-01-26 17:00:33.626428,-2.0,-0.000324,23.350200,0.165493,0.291256,"Option(conId=671974250, symbol='IAU', lastTrad...",-33.10,13.60,0.0,U7706434
2,IAU,,STK,2024-01-26 17:00:33.626411,300.0,0.112001,38.454255,38.163998,-0.007548,"Stock(conId=474829650, symbol='IAU', right='0'...",11449.20,-87.08,0.0,U7706434
3,INDA,,STK,2024-01-26 17:00:33.626446,100.0,0.048192,49.712499,49.263996,-0.009022,"Stock(conId=101484826, symbol='INDA', right='0...",4926.40,-44.85,0.0,U7706434
4,IUSQ,,STK,2024-01-26 17:00:33.626462,100.0,0.069002,65.602785,70.536720,0.075209,"Stock(conId=239363241, symbol='IUSQ', right='0...",7053.67,493.39,0.0,U7706434
5,IWM,,Call 199.0 20240216,2024-01-26 17:00:33.626479,-1.0,-0.003365,183.368900,3.439353,-0.875647,"Option(conId=637114226, symbol='IWM', lastTrad...",-343.94,-160.57,0.0,U7706434
6,IWM,,STK,2024-01-26 17:00:33.626471,100.0,0.192558,199.616920,196.839996,-0.013911,"Stock(conId=9579970, symbol='IWM', right='0', ...",19684.00,-277.69,0.0,U7706434
7,PRX,,STK,2024-01-26 17:00:33.626490,250.0,0.069222,26.561554,28.304394,0.065615,"Stock(conId=382625193, symbol='PRX', right='0'...",7076.10,435.71,0.0,U7706434
8,SOXX,,Put 525.0 20240216,2024-01-26 17:00:33.626499,-1.0,-0.000569,699.944800,0.581455,0.916928,"Option(conId=653437229, symbol='SOXX', lastTra...",-58.15,641.80,0.0,U7706434
9,TLT,,Call 96.0 20240216,2024-01-26 17:00:33.626524,-1.0,-0.000518,76.199700,0.530000,0.304459,"Option(conId=614854428, symbol='TLT', lastTrad...",-53.00,23.20,0.0,U7706434


# Add some data to portfolio

In [3]:
ac = initialize_db('db')
ac.list_libraries()
# general = ac.get_library('general')
portfolio_lib = ac.get_library('portfolio')
portfolio_lib.list_symbols()

['positions']

In [29]:
dummy_data = df.loc[0,:].copy()
dummy_data['position'] = dummy_data['position'] / 2
dummy_data['% of nav'] = dummy_data['% of nav'] / 2
dummy_data['averageCost'] = dummy_data['averageCost'] * 0.9
dummy_data['unrealizedPNL'] = (dummy_data['marketPrice'] - dummy_data['averageCost']) * dummy_data['position']
dummy_data['marketValue'] = dummy_data['marketPrice'] * dummy_data['position']
if dummy_data['asset class'] == 'STK' or dummy_data['asset class'] == 'FUT':
    pnl = ((dummy_data.marketPrice/(dummy_data.averageCost)) -1)
    pnl = pnl *(-1) if dummy_data.position < 0 else pnl
else:
    if dummy_data.position < 0:
        pnl = ((dummy_data.marketPrice/(dummy_data.averageCost/100)) -1) * (-1)
    else:
        pnl = ( (dummy_data.marketPrice/ (dummy_data.averageCost/100)) -1)
dummy_data['pnl %'] = pnl
dummy_data = pd.DataFrame(dummy_data).T


In [31]:
# Convert 'contract' column to string as ArcticDB may not handle custom objects well
dummy_data['contract'] = dummy_data['contract'].astype(str)

# Ensure that all other columns are of a consistent and compatible data type
# For example, convert all numeric columns to float
numeric_columns = ['position', '% of nav', 'averageCost', 'marketPrice', 'pnl %', 'marketValue', 'unrealizedPNL', 'realizedPNL']
for col in numeric_columns:
    dummy_data[col] = pd.to_numeric(dummy_data[col], errors='coerce')

# Now write the modified DataFrame to ArcticDB
portfolio_lib.write('positions', dummy_data)


VersionedItem(symbol='positions', library='portfolio', data=n/a, version=0, metadata=None, host='S3(endpoint=s3.eu-central-1.amazonaws.com, bucket=lowquant-arcticdb)')

In [16]:
df_ac = portfolio_lib.read('positions').data
df_ac

,timestamp,symbol,asset class,position,% of nav,averageCost,marketPrice,pnl %,strategy,contract,marketValue,unrealizedPNL,realizedPNL,account
0,2024-01-21 15:27:35.463259,EWT,STK,100.0,0.04393,39.831146,44.210701,0.109953,,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


#### Now, we will get ib positions and match them with arcticdb entries

In [18]:
df_ib = pm.get_positions_from_ib()

In [ ]:
def match_ib_positions_with_arcticdb():
    library = ac.get_library('portfolio',create_if_missing=True)
    df_ac = library.read('positions').data
    latest_positions = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
    df_ib =  df_ib # self.get_positions_from_ib()

In [19]:
df_ac = portfolio_lib.read('positions').data
latest_positions_in_ac = df_ac.sort_values(by='timestamp').groupby(['symbol', 'strategy', 'asset class']).last().reset_index()
latest_positions_in_ac

,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.04393,39.831146,44.210701,0.109953,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.0701,437.955535,0.0,U7706434


In [ ]:
def create_new_row():
    new_row = { 'timestamp': datetime.datetime.now().strftime('%Y-%m-%d %H:%M'),
                'symbol': symbol,'asset class': asset_class,
                'position':item.position,
                '% of nav':item.marketValue/total_equity,
                'averageCost': item.averageCost,
                'marketPrice': item.marketPrice,
                'pnl %': pnl,
                'strategy': '',
                'contract': str(item.contract),
                'marketValue': item.marketValue,
                'unrealizedPNL': item.unrealizedPNL,
                'realizedPNL': item.realizedPNL,
                'account': item.account}

In [39]:
df_merged = pd.DataFrame()
total_equity =  sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_entries_in_ac = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]
    
    if not strategy_entries_in_ac.empty:
        strategy_entry = strategy_entries_in_ac.iloc[0]
        if total_position == strategy_entry.position:
            # Since the positions match, we use the latest data from ib and only add the strategy field
            row['strategy'] = strategy_entry.strategy
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            for entry in strategy_entries_in_ac:
                # update fields in: ['marketPrice', '% of nav','pnl %']
                strategy_entries_in_ac.loc[entry,'% of nav'] = ((strategy_entries_in_ac[entry,'position'] / total_position) * row['marketValue']) / total_equity
                # other update fields here

                # add entry to df_merged
            # ---------------------------------------------
                    
            # Calculate the averageCost, '% of nav','pnl %' and position for the residual
            assigned_position = strategy_entries_in_ac['position'].sum()
            residual_position = total_position - assigned_position

            # Calculate the average cost for the residual position
            ib_avg_cost = row['averageCost']
            ac_avg_cost = strategy_entries_in_ac.apply(lambda x: x['averageCost'] * (x['position'] / total_position), axis=1).sum()
            residual_avg_cost = (ib_avg_cost - ac_avg_cost) * (total_position / residual_position)
            #print(f"{symbol:} ib_avg_cost: {ib_avg_cost}, residual avg cost {residual_avg_cost}, ac_avg_cost {strategy_entries_in_ac.averageCost}")

            # do other calculations

            # concat residual position to df_merged
            
    else:
    # we have positions that are not in arcticdb that we should display & save as well
        pass
        

df_merged
    

EWT ib_avg_cost: 44.2568285, residual avg cost 48.68251135, ac_avg_cost 0    39.831146
Name: averageCost, dtype: float64


""


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu allen Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; fundfarm; ushmds; secdefeu.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wu

In [54]:
import pandas as pd

# Assuming df_ib is your DataFrame from ib.positions()
# Assuming latest_positions_in_ac is your DataFrame from ArcticDB with the latest position entries

df_merged = pd.DataFrame()
total_equity = sum(float(entry.value) for entry in ib.accountSummary() if entry.tag == "EquityWithLoanValue")

# Iterate through the positions obtained from Interactive Brokers
for index, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    
    # Filter the ArcticDB DataFrame for entries with the same symbol and asset class
    strategy_entries_in_ac = latest_positions_in_ac[
        (latest_positions_in_ac['symbol'] == symbol) & 
        (latest_positions_in_ac['asset class'] == asset_class)
    ]
    
    if not strategy_entries_in_ac.empty:
        # Calculate the total position assigned to strategies in ArcticDB
        assigned_position = strategy_entries_in_ac['position'].sum()
        
        # If the total position from IB matches the sum of assigned positions in ArcticDB
        if total_position == assigned_position:
            # Use the row data and add the strategy info
            row['strategy'] = strategy_entries_in_ac['strategy'].values[0]  # Assuming one strategy per symbol/asset class
            df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)
        else:
            # There's a discrepancy between the positions, so we need to handle the residual
            # First, update the existing entries from ArcticDB
            strategy_entries_in_ac['marketPrice'] = row['marketPrice']
            strategy_entries_in_ac['% of nav'] = (strategy_entries_in_ac['position'] * row['marketPrice']) / total_equity
            strategy_entries_in_ac['pnl %'] = (strategy_entries_in_ac['marketPrice'] / strategy_entries_in_ac['averageCost'] - 1) * 100
            
            # Append the updated entries to df_merged
            df_merged = pd.concat([df_merged, strategy_entries_in_ac], ignore_index=True)
            
            # Now, calculate and append the residual position entry
            residual_position = total_position - assigned_position
            residual_avg_cost = ((row['averageCost'] * total_position) - (strategy_entries_in_ac['averageCost'] * assigned_position)) / residual_position
            residual_avg_cost = residual_avg_cost[0]
            residual_market_value = residual_position * row['marketPrice']
            residual_percent_nav = residual_market_value / total_equity
            residual_pnl_percent = ((row['marketPrice'] / residual_avg_cost) - 1) * 100
            
            residual_entry = row.copy()
            residual_entry['position'] = residual_position
            residual_entry['averageCost'] = residual_avg_cost
            residual_entry['marketValue'] = residual_market_value
            residual_entry['% of nav'] = residual_percent_nav
            residual_entry['pnl %'] = residual_pnl_percent
            residual_entry['strategy'] = ''  # No strategy assigned yet
            
            df_merged = pd.concat([df_merged, pd.DataFrame([residual_entry])], ignore_index=True)
            
    else:
        # If there are no entries in ArcticDB for this symbol/asset class, just append the row from ib
        df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

df_merged


,symbol,strategy,asset class,timestamp,position,% of nav,averageCost,marketPrice,pnl %,contract,marketValue,unrealizedPNL,realizedPNL,account
0,EWT,,STK,2024-01-21 15:27:35.463259,100.0,0.044055,39.831146,45.020000,13.027129,"Stock(conId=253190576, symbol='EWT', right='0'...",4421.070100,437.955535,0.0,U7706434
1,EWT,,STK,2024-01-26 21:23,100.0,0.044055,48.682511,45.020000,-7.523258,"Stock(conId=253190576, symbol='EWT', right='0'...",4502.000045,152.630000,0.0,U7706434
2,IAU,,STK,2024-01-26 21:23,300.0,0.112103,38.454255,38.183998,-0.007028,"Stock(conId=474829650, symbol='IAU', right='0'...",11455.200000,-81.080000,0.0,U7706434
3,IAU,,Call 39.0 20240216,2024-01-26 21:23,-2.0,-0.000324,23.350200,0.165493,0.291256,"Option(conId=671974250, symbol='IAU', lastTrad...",-33.100000,13.600000,0.0,U7706434
4,INDA,,STK,2024-01-26 21:23,100.0,0.048095,49.712499,49.146000,-0.011396,"Stock(conId=101484826, symbol='INDA', right='0...",4914.600000,-56.650000,0.0,U7706434
5,IUSQ,,STK,2024-01-26 21:23,100.0,0.068934,65.602785,70.440002,0.073735,"Stock(conId=239363241, symbol='IUSQ', right='0...",7044.000000,483.720000,0.0,U7706434
6,IWM,,STK,2024-01-26 21:23,100.0,0.191716,199.616920,195.903992,-0.018600,"Stock(conId=9579970, symbol='IWM', right='0', ...",19590.400000,-371.290000,0.0,U7706434
7,IWM,,Call 199.0 20240216,2024-01-26 21:23,-1.0,-0.002821,183.368900,2.883058,-0.572272,"Option(conId=637114226, symbol='IWM', lastTrad...",-288.310000,-104.940000,0.0,U7706434
8,PRX,,STK,2024-01-26 21:23,250.0,0.069140,26.561554,28.260000,0.063944,"Stock(conId=382625193, symbol='PRX', right='0'...",7065.000000,424.610000,0.0,U7706434
9,SOXX,,Put 525.0 20240216,2024-01-26 21:23,-1.0,-0.000827,699.944800,0.844985,0.879278,"Option(conId=653437229, symbol='SOXX', lastTra...",-84.500000,615.450000,0.0,U7706434


Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde wiederhergestellt \u2013 Daten sind erhalten geblieben. Verbindung zu folgenden Datenzentren hergestellt: usfarm.nj; eufarm; usopt; usfarm; euhmds; secdefeu. Keine Verbindung zu folgenden Datenzentren: fundfarm; ushmds.
Error 1100, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation ist abgebrochen.
Error 1102, reqId -1: Verbindung zwischen %SHORT:COMPNAME% und Trader Workstation wurde

In [48]:
latest_positions_in_ac.iloc[0]['contract']

AttributeError: 'str' object has no attribute 'to_dict'

In [33]:
import pandas as pd

df_merged = pd.DataFrame()  # Ensure df_merged is initialized as an empty DataFrame

# ... your code ...

for index, row in df_ib.iterrows():
    # ... your code ...
    df_merged = pd.concat([df_merged, pd.DataFrame([row])], ignore_index=True)

# ... your code ...


In [35]:
df_merged.append(row)

AttributeError: 'DataFrame' object has no attribute 'append'

In [22]:
# Assuming df_ib and latest_positions_in_ac are your DataFrames from IB and ArcticDB

for _, row in df_ib.iterrows():
    symbol = row['symbol']
    asset_class = row['asset class']
    total_position = row['position']
    # total_average_cost = row['averageCost']
    # total_market_price = row['marketPrice']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & 
                                           (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        assigned_position = strategy_rows['position'].sum()
        residual_position = total_position - assigned_position

        if len(strategy_rows) == 1 and total_position == strategy_rows.iloc[0]['position']:
            # Case 1: Single strategy row and matching position
            # Update all columns in ArcticDB for this symbol & asset class combination except 'strategy'
            # Example: Update marketPrice and other related fields in ArcticDB
            pass

        else:
            # Case 2: Single/Multiple strategy rows and differing position
            # Calculate averageCost for the residual position
            assigned_average_cost = strategy_rows['averageCost'].iloc[0] if len(strategy_rows) == 1 else calculate_weighted_average_cost(strategy_rows)
            residual_average_cost = calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position)

            # Update marketPrice, marketValue, % of NAV, pnl%, unrealizedPNL for the assigned positions in ArcticDB
            # And calculate the corresponding values for the residual position
            pass

def calculate_weighted_average_cost(strategy_rows):
    # Implement logic to calculate a weighted average cost for multiple strategy rows
    pass

def calculate_residual_average_cost(total_average_cost, total_position, assigned_average_cost, assigned_position):
    # Implement logic to calculate the average cost for the residual position
    return ((total_average_cost * total_position) - (assigned_average_cost * assigned_position)) / (total_position - assigned_position)

# Further implement the functions or logic needed in 'pass' sections

1


In [57]:
for _, row in df_ib.iterrows():
    symbol = row.symbol
    asset_class = row['asset class']
    total_position = row['position']

    strategy_rows = latest_positions_in_ac[(latest_positions_in_ac['symbol'] == symbol) & (latest_positions_in_ac['asset class'] == asset_class)]

    if not strategy_rows.empty:
        if len(strategy_rows) == 1 and total_position == strategy_rows.position:
            # update all cols in ac for this symbol & asset class combination except col strategies
            pass
        if len(strategy_rows) == 1 and total_position != strategy_rows.position:
            # 1. update cols marketPrice in ac from ib query and calculate new values for 'marketValue' (position*marketPrice), '% of nav', 'pnl%','unrealized'
            # calculate averageCost for the residual position (ib total averageCost = ac.averageCost * (ac.position/total_position) + residual.averageCost * (residual.position/total_position))
            # with the averageCost value and the position for the residual we can calculate 'marketValue', '% of nav', 'pnl%','unrealized'

            # if possible generalize the above for the cases where len(strategy_rows) > 1
            pass


1
EWT in ac
100.0 100.0


In [42]:
latest_positions['symbol'].values

array(['EWT'], dtype=object)